In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

spark


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/21 11:29:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

schema = StructType([
    StructField("Days_for_shipment_scheduled", IntegerType(), True),
    StructField("Benefit_per_order", DoubleType(), True),

    StructField("Sales_per_customer", DoubleType(), True),
    StructField("Delivery_Status", StringType(), True),
    StructField("Customer_Country", StringType(), True),
    StructField("Customer_City", StringType(), True),
    StructField("Order_date", StringType(), True)
])

allcol = spark.readStream.format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

df_stream = allcol.select(from_json(col("value"), schema).alias("data")).select("data.*")


print(df_stream)

25/11/21 11:29:10 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


DataFrame[Days_for_shipment_scheduled: int, Benefit_per_order: double, Sales_per_customer: double, Delivery_Status: string, Customer_Country: string, Customer_City: string, Order_date: string]


In [3]:
df_stream.printSchema()

root
 |-- Days_for_shipment_scheduled: integer (nullable = true)
 |-- Benefit_per_order: double (nullable = true)
 |-- Sales_per_customer: double (nullable = true)
 |-- Delivery_Status: string (nullable = true)
 |-- Customer_Country: string (nullable = true)
 |-- Customer_City: string (nullable = true)
 |-- Order_date: string (nullable = true)



In [4]:
query = df_stream.writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()

25/11/21 11:29:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1062d683-50b9-4167-991e-23f8d0242b68. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/11/21 11:29:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/11/21 11:29:12 ERROR MicroBatchExecution: Query [id = b3071964-21b3-435d-a12a-2bc50dd10f2e, runId = e716929c-c131-4716-89e4-786e84ebd4be] terminated with error
java.net.ConnectException: Connection refused
	at java.base/sun.nio.ch.Net.connect0(Native Method)
	at java.base/sun.nio.ch.Net.connect(Net.java:589)
	at java.base/sun.nio.ch.Net.connect(Net.java:578)
	at java.base/sun.nio.ch.NioSocketImpl.connect(NioSocketImpl.java:583)
	at java.base/java.net.SocksSocketIm

StreamingQueryException: [STREAM_FAILED] Query [id = b3071964-21b3-435d-a12a-2bc50dd10f2e, runId = e716929c-c131-4716-89e4-786e84ebd4be] terminated with exception: Connection refused SQLSTATE: XXKST
=== Streaming Query ===
Identifier: [id = b3071964-21b3-435d-a12a-2bc50dd10f2e, runId = e716929c-c131-4716-89e4-786e84ebd4be]
Current Committed Offsets: {}
Current Available Offsets: {TextSocketV2[host: localhost, port: 9999]: -1}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
~WriteToMicroBatchDataSource org.apache.spark.sql.execution.streaming.ConsoleTable$@5822b96f, b3071964-21b3-435d-a12a-2bc50dd10f2e, [truncate=false], Append
+- ~Project [data#1.Days_for_shipment_scheduled AS Days_for_shipment_scheduled#2, data#1.Benefit_per_order AS Benefit_per_order#3, data#1.Sales_per_customer AS Sales_per_customer#4, data#1.Delivery_Status AS Delivery_Status#5, data#1.Customer_Country AS Customer_Country#6, data#1.Customer_City AS Customer_City#7, data#1.Order_date AS Order_date#8]
  